<a href="https://colab.research.google.com/github/fabrito2006/Laboratorio-15-MID/blob/main/Laboratorio_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 15 - Mineria de Datos
## Alumno: Torres Alvaradao Fabricio Ismael
## Ciclo y carrera: 5C28A

## 1. Aplicaremos los principales conceptos sobre minería de textos haciendo uso de la librería ‘chromadb’ para implementar una base de datos vectorial en Python (https://pypi.org/project/chromadb/)

In [ ]:
!pip install chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.2 MB/s eta 

## 2. Una vez instaladas, procedemos a importarlas.

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

## 3. Lo que haremos a continuación instanciar una base de datos local y crear una colección (tabla) de nombre “alumnos”. De la misma manera, debemos escoger un motor de embeddings para convertir un texto a vector.

In [ ]:
# Inicializar el motor de vectores local
client = chromadb.Client()

# Crear colección (equivalente a una tabla)
collection = client.create_collection(name="alumnos")
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 4. Ahora, vamos a crear unas entradas de texto para convertirlos en embeddings y almacenarlos en la base de datos

In [ ]:
# Lista de documentos de ejemplo
docs = [
    "Luis es un excelente alumno de ciencia de datos",
    "María tiene interés en la inteligencia artificial",
    "Pedro trabaja en procesamiento de lenguaje natural"
]

# Generar vectores
embeddings = model.encode(docs).tolist()

# Insertar en la colección
collection.add(
    documents=docs,
    embeddings=embeddings,
    ids=["1", "2", "3"]
)

## 5. Ahora, procederemos a colocar un prompt cualquiera y haremos una selección hacia nuestra base de datos vectorial

In [ ]:
query = "¿Quién es un alumno sobresaliente?"

query_embedding = model.encode([query]).tolist()

# Buscar los 2 más cercanos
results = collection.query(
    query_embeddings=query_embedding,
    n_results=2
)

print(results['documents'])

[['Pedro trabaja en procesamiento de lenguaje natural', 'Luis es un excelente alumno de ciencia de datos']]


## 6. Finalmente, vamos a visualizar como el modelo convirtió nuestro texto en vectores

In [ ]:
data = collection.get(include=["embeddings", "documents"])

for doc, id_, emb in zip(data['documents'], data['ids'], data['embeddings']):
    print(f"ID: {id_}")
    print(f"Documento: {doc}")
    print(f"Embedding (primeras 5 dimensiones): {emb[:5]}")
    print("-" * 30)

ID: 1
Documento: Luis es un excelente alumno de ciencia de datos
Embedding (primeras 5 dimensiones): [-0.0515781   0.07000642 -0.04834177 -0.05982044 -0.01550355]
------------------------------
ID: 2
Documento: María tiene interés en la inteligencia artificial
Embedding (primeras 5 dimensiones): [-0.03857211 -0.01005658  0.00841351 -0.03409148 -0.09196933]
------------------------------
ID: 3
Documento: Pedro trabaja en procesamiento de lenguaje natural
Embedding (primeras 5 dimensiones): [ 0.04198885  0.0491558  -0.02113147  0.01458174 -0.09477768]
------------------------------


## 7. Si quieres visualizar lo que almacena una base de datos vectorial, realizar la siguiente acción

In [ ]:
data = collection.get()
print(data)


{'ids': ['1', '2', '3'], 'embeddings': None, 'documents': ['Luis es un excelente alumno de ciencia de datos', 'María tiene interés en la inteligencia artificial', 'Pedro trabaja en procesamiento de lenguaje natural'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [None, None, None]}


## Parte Final

In [ ]:
collection_minilm = client.get_or_create_collection(name="Tecsup_minilm")
print("Colección 'Tecsup_minilm' creada ")

Colección 'Tecsup_minilm' creada o obtenida con éxito.


In [ ]:
# Información de ejemplo
personal_info = [
    {"id": "1", "name": "Juan Pérez", "role": "Estudiante de IA", "hobbies": "Programación, videojuegos"},
    {"id": "2", "name": "Ana García", "role": "Estudiante de Ciberseguridad", "hobbies": "Cine, lectura"},
    {"id": "3", "name": "Carlos Rodríguez", "role": "Profesor de Redes", "hobbies": "Senderismo, cocina"},
    {"id": "4", "name": "Sofía Vargas", "role": "Estudiante de Diseño Gráfico", "hobbies": "Dibujo, fotografía"},
    {"id": "5", "name": "Martín Sánchez", "role": "Estudiante de Desarrollo de Software", "hobbies": "Música, deportes"},
    {"id": "6", "name": "Luisa Torres", "role": "Coordinadora Académica", "hobbies": "Viajar, jardinería"},
    {"id": "7", "name": "Diego Castro", "role": "Estudiante de IoT", "hobbies": "Robótica, ajedrez"},
    {"id": "8", "name": "Elena Morales", "role": "Estudiante de Ciencia de Datos", "hobbies": "Análisis de datos, escribir"},
]

# Convertir la información a un formato de documentos para el embedding
documents_minilm = [
    f"{info['name']} es {info['role']} y sus hobbies son {info['hobbies']}."
    for info in personal_info
]
ids_minilm = [info['id'] for info in personal_info]
metadatas_minilm = [{"name": info['name'], "role": info['role']} for info in personal_info]


In [ ]:
print("\nCargando el primer modelo de embedding (all-MiniLM-L6-v2)...")
model_minilm = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings
print("Generando embeddings para la información personal...")
embeddings_minilm = model_minilm.encode(documents_minilm).tolist()

# Añadir los documentos a la colección
print("Añadiendo documentos a la colección 'Tecsup_minilm'...")
collection_minilm.add(
    documents=documents_minilm,
    embeddings=embeddings_minilm,
    metadatas=metadatas_minilm,
    ids=ids_minilm
)
print(f"Número de documentos en la colección 'Tecsup_minilm': {collection_minilm.count()}")


Cargando el primer modelo de embedding (all-MiniLM-L6-v2)...
Generando embeddings para la información personal...
Añadiendo documentos a la colección 'Tecsup_minilm'...
Número de documentos en la colección 'Tecsup_minilm': 8


In [ ]:
def query_tecsup_minilm(query_text, n_results=2):
    """
    Función para realizar consultas a la base de datos vectorial Tecsup
    usando el modelo all-MiniLM-L6-v2.
    """
    query_embedding = model_minilm.encode([query_text]).tolist()

    results = collection_minilm.query(
        query_embeddings=query_embedding,
        n_results=n_results,
        include=['documents', 'metadatas', 'distances']
    )
    return results

print("\n--- Consultas a la base de datos 'Tecsup_minilm' ---")


--- Consultas a la base de datos 'Tecsup_minilm' ---


In [ ]:
query1 = "¿Quién le gusta programar y jugar videojuegos?"
results1 = query_tecsup_minilm(query1)
print(f"\nConsulta: \"{query1}\"")
for i, doc in enumerate(results1['documents'][0]):
    print(f"  Resultado {i+1}: {doc}")
    print(f"    ID: {results1['ids'][0][i]}")
    print(f"    Nombre: {results1['metadatas'][0][i]['name']}")
    print(f"    Distancia: {results1['distances'][0][i]:.4f}")


query2 = "¿Qué profesiones hay en Tecsup relacionadas con la seguridad?"
results2 = query_tecsup_minilm(query2, n_results=3)
print(f"\nConsulta: \"{query2}\"")
for i, doc in enumerate(results2['documents'][0]):
    print(f"  Resultado {i+1}: {doc}")
    print(f"    ID: {results2['ids'][0][i]}")
    print(f"    Nombre: {results2['metadatas'][0][i]['name']}")
    print(f"    Distancia: {results2['distances'][0][i]:.4f}")

query3 = "¿Quién es un profesor y qué hace en su tiempo libre?"
results3 = query_tecsup_minilm(query3)
print(f"\nConsulta: \"{query3}\"")
for i, doc in enumerate(results3['documents'][0]):
    print(f"  Resultado {i+1}: {doc}")
    print(f"    ID: {results3['ids'][0][i]}")
    print(f"    Nombre: {results3['metadatas'][0][i]['name']}")
    print(f"    Distancia: {results3['distances'][0][i]:.4f}")



Consulta: "¿Quién le gusta programar y jugar videojuegos?"
  Resultado 1: Juan Pérez es Estudiante de IA y sus hobbies son Programación, videojuegos.
    ID: 1
    Nombre: Juan Pérez
    Distancia: 0.5507
  Resultado 2: Martín Sánchez es Estudiante de Desarrollo de Software y sus hobbies son Música, deportes.
    ID: 5
    Nombre: Martín Sánchez
    Distancia: 0.9028

Consulta: "¿Qué profesiones hay en Tecsup relacionadas con la seguridad?"
  Resultado 1: Martín Sánchez es Estudiante de Desarrollo de Software y sus hobbies son Música, deportes.
    ID: 5
    Nombre: Martín Sánchez
    Distancia: 1.1374
  Resultado 2: Elena Morales es Estudiante de Ciencia de Datos y sus hobbies son Análisis de datos, escribir.
    ID: 8
    Nombre: Elena Morales
    Distancia: 1.1631
  Resultado 3: Juan Pérez es Estudiante de IA y sus hobbies son Programación, videojuegos.
    ID: 1
    Nombre: Juan Pérez
    Distancia: 1.1679

Consulta: "¿Quién es un profesor y qué hace en su tiempo libre?"
  Resulta